# Raw Dataset 처리 코드

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 대한적십자사로부터 제공받은 데이터 처리

##### -> 붙임 2. 정보공개 청구 회신자료 서식(병원별 혈액 청구량) (1) (1).xlsx

In [2]:
import pandas as pd

# 파일 경로
file_path = "/content/drive/MyDrive/co-data/붙임 2. 정보공개 청구 회신자료 서식(병원별 혈액 청구량).xlsx"

# 엑셀 파일 읽기 (첫 번째 시트 기준)
df = pd.read_excel(file_path)

In [3]:
# 출고일자 열을 datetime 형식으로 변환
df['출고일자'] = pd.to_datetime(df['출고일자'], errors='coerce')

# 변수 이름을 영어로 변경
df = df.rename(columns={'출고병원': '기관명'})

# 날짜 관련 열 확인 (예: '청구일자' 또는 '날짜'와 같은 열이 있어야 함)
print(df.head())

        출고일자                기관명  출고건수
0 2020-01-01  (성서본원)계명대학교 동산의료원    61
1 2020-01-01  (의) 한라의료재단 제주한라병원    33
2 2020-01-01        (칠곡)경북대학교병원     6
3 2020-01-01             BS종합병원     2
4 2020-01-01      CHA의과대학교분당차병원    70


병원 Unique 개수

In [4]:
# 병원 개수 확인
num_hospitals = df['기관명'].nunique()

print(f"총 병원 개수: {num_hospitals}")

총 병원 개수: 3172


### 서울시로부터 제공받은 데이터 처리

##### -> 서울시 병의원 위치 정보.csv

1. '기관명', '병원경도', '병원위도' 만 추출

In [5]:
import pandas as pd

# 파일 경로
file_path = "/content/drive/MyDrive/co-data/서울시 병의원 위치 정보.csv"

# CSV 파일 읽기 (인코딩 문제 발생 시 encoding='utf-8' 또는 'cp949' 추가)
df_hospitals = pd.read_csv(file_path, encoding='cp949')

# 필요한 열만 선택
df_selected = df_hospitals[['기관명', '병원경도', '병원위도']]

# 결과 확인
print(df_selected.head())

# 선택한 데이터 저장 (선택 사항)
output_path = "/content/drive/MyDrive/co-data/seoul_hospitals_selected.csv"
df_selected.to_csv(output_path, index=False, encoding='utf-8')

print(f"필터링된 데이터가 {output_path}에 저장되었습니다.")

                      기관명        병원경도       병원위도
0                 가산기대찬의원  126.884122  37.480394
1                 365엠씨의원  127.021809  37.516604
2  (사)한국한센복지협회서울특별시지부부설의원  126.938038  37.504076
3           (의)가산의료재단광동병원  127.062147  37.514281
4                 가로수치과의원  126.929377  37.481918
필터링된 데이터가 /content/drive/MyDrive/co-data/seoul_hospitals_selected.csv에 저장되었습니다.


2. 총 병원 Unique 개수

In [6]:
# 병원(기관명) 개수 확인
num_hospitals = df_selected['기관명'].nunique()

print(f"총 병원 개수: {num_hospitals}")

총 병원 개수: 17372


#### 서울시 병원 데이터 병원 중복값 처리

In [7]:
import pandas as pd

# 데이터 파일 경로
file_path = "/content/drive/MyDrive/co-data/seoul_hospitals_selected.csv"

# 데이터 로드
try:
    # 인코딩 이슈가 있을 수 있으므로 utf-8과 cp949를 모두 시도
    try:
        df = pd.read_csv(file_path, encoding='utf-8')
    except UnicodeDecodeError:
        df = pd.read_csv(file_path, encoding='cp949')

    # 컬럼명 표준화 (위도, 경도 컬럼 확인)
    column_mapping = {
        '병원경도': '경도',
        '병원위도': '위도'
    }
    df.rename(columns=column_mapping, inplace=True)

    # 기관명 컬럼이 있는지 확인
    if '기관명' in df.columns and '위도' in df.columns and '경도' in df.columns:
        # 중복 제거: 기관명, 위도, 경도가 모두 같은 경우 하나만 남김
        df = df.drop_duplicates(subset=['기관명', '위도', '경도'])

        # 기관명 중복 개수 계산
        duplicates_count = df['기관명'].duplicated().sum()
        print(f'중복된 기관명 개수: {duplicates_count}개')

        # 중복을 포함한 모든 기관명 개수
        total_count = df['기관명'].count()
        print(f'전체 기관명 개수: {total_count}개')

        # 유니크한 기관명 개수
        unique_count = df['기관명'].nunique()
        print(f'유니크한 기관명 개수: {unique_count}개')

        # 중복된 기관명들과 그 개수 출력
        print("\n중복된 기관명과 개수:")
        duplicated_institutions = df['기관명'].value_counts()
        for name, count in duplicated_institutions[duplicated_institutions > 1].items():
            print(f"{name}: {count}개")
    else:
        # 다른 이름의 컬럼이 기관명일 수 있으므로 컬럼 목록 출력
        print("'기관명' 컬럼을 찾을 수 없습니다. 다음은 이 CSV 파일의 컬럼 목록입니다:")
        print(df.columns.tolist())

        # 가능한 기관명 컬럼 추측
        possible_columns = [col for col in df.columns if '기관' in col or '병원' in col or '이름' in col or 'name' in col.lower()]
        if possible_columns:
            print("\n다음 컬럼들이 기관명을 나타낼 수 있습니다:")
            for col in possible_columns:
                print(f"- {col}")
except Exception as e:
    print(f"파일을 로드하는 중 오류가 발생했습니다: {e}")
    print("파일이 존재하고 접근 가능한지 확인해주세요.")

중복된 기관명 개수: 3867개
전체 기관명 개수: 21239개
유니크한 기관명 개수: 17372개

중복된 기관명과 개수:
연세이비인후과의원: 22개
경희한의원: 18개
서울삼성내과의원: 16개
연세치과의원: 16개
연세내과의원: 14개
유디치과의원: 14개
연세안과의원: 14개
한사랑의원: 14개
화인마취통증의학과의원: 13개
서울수치과의원: 12개
함소아한의원: 12개
연세정형외과의원: 12개
서울이비인후과의원: 12개
메이퓨어의원: 12개
서울미소치과의원: 12개
서울치과의원: 12개
연세재활의학과의원: 12개
우리치과의원: 12개
아비쥬의원: 12개
이사랑치과의원: 11개
굿모닝치과의원: 11개
서울안과의원: 11개
서울가정의학과의원: 11개
서울본치과의원: 11개
서울열린치과의원: 11개
미소치과의원: 10개
성모안과의원: 10개
서울하이안치과의원: 10개
서울정형외과의원: 10개
삼성키즈소아청소년과의원: 10개
세종한의원: 10개
삼성드림소아청소년과의원: 10개
밴스의원: 10개
서울경희한의원: 10개
정치과의원: 10개
행복한치과의원: 10개
웰치과의원: 10개
블리비의원: 10개
프라임치과의원: 10개
맑은숲한의원: 9개
수한의원: 9개
고운이치과의원: 9개
애플산부인과의원: 9개
청구경희한의원: 9개
연치과의원: 9개
연세가정의학과의원: 9개
제일한의원: 9개
서울의원: 9개
서울미치과의원: 9개
우리정형외과의원: 9개
두리이비인후과의원: 9개
서울내과의원: 9개
박치과의원: 9개
아름다운치과의원: 9개
하나치과의원: 9개
동의보감한의원: 9개
명인한의원: 9개
하나의원: 9개
미치과의원: 8개
일맥한의원: 8개
상쾌한이비인후과의원: 8개
사랑의치과의원: 8개
연세수치과의원: 8개
최치과의원: 8개
미앤미의원: 8개
이내과의원: 8개
제일치과의원: 8개
이편한치과의원: 8개
스마일치과의원: 8개
하늘마음한의원: 8개
평강한의원: 8개
연세의원: 8개
세란치과의원: 8개
서울삼성치과의원: 8개
소생한의원: 8개
윤치과의원: 8개
조은치과의원: 

중복된 병원 총 출고건수

In [8]:
import pandas as pd

# 데이터 파일 경로
seoul_hospitals_path = "/content/drive/MyDrive/co-data/seoul_hospitals_selected.csv"
blood_release_path = "/content/drive/MyDrive/co-data/붙임 2. 정보공개 청구 회신자료 서식(병원별 혈액 청구량).xlsx"

# 데이터 로드
try:
    # 서울 병원 데이터 로드 (인코딩 처리)
    try:
        seoul_df = pd.read_csv(seoul_hospitals_path, encoding='utf-8')
    except UnicodeDecodeError:
        seoul_df = pd.read_csv(seoul_hospitals_path, encoding='cp949')

    # 혈액 방출 데이터 로드
    blood_df = pd.read_excel(blood_release_path)

    # 컬럼명 표준화 (위도, 경도 컬럼 확인)
    column_mapping = {
        '병원경도': '경도',
        '병원위도': '위도'
    }
    seoul_df.rename(columns=column_mapping, inplace=True)

    # 기관명 컬럼 확인 및 설정
    institution_col = '기관명' if '기관명' in seoul_df.columns else next(
        (col for col in seoul_df.columns if '기관' in col or '병원' in col or '이름' in col or 'name' in col.lower()), None
    )

    hospital_col = '기관명' if '기관명' in blood_df.columns else next(
        (col for col in blood_df.columns if '병원' in str(col) or 'hospital' in str(col).lower() or '기관' in str(col)), None
    )

    if institution_col is None or hospital_col is None:
        raise ValueError("기관명 또는 병원명이 포함된 컬럼을 찾을 수 없습니다.")

    # 공백 제거 후 새로운 컬럼 생성
    seoul_df['clean_name'] = seoul_df[institution_col].astype(str).str.replace(' ', '', regex=True)
    blood_df['clean_hospital'] = blood_df[hospital_col].astype(str).str.replace(' ', '', regex=True)

    # 중복 제거: 기관명, 위도, 경도가 모두 같은 경우 하나만 유지
    seoul_df = seoul_df.drop_duplicates(subset=['clean_name', '위도', '경도'])

    # 서울 병원 데이터에서 중복된 기관명 찾기
    duplicated_seoul = seoul_df[seoul_df['clean_name'].duplicated(keep=False)]
    duplicated_names = duplicated_seoul['clean_name'].unique()
    print(f"서울 병원 데이터에서 중복된 기관명 수: {len(duplicated_names)}개")

    # 혈액 방출 데이터에서 중복된 기관명 확인 및 출고 건수 합산
    blood_matched = blood_df[blood_df['clean_hospital'].isin(duplicated_names)]

    if not blood_matched.empty:
        # 출고 건수 컬럼 자동 탐색 (숫자형 컬럼 찾기)
        numeric_cols = blood_matched.select_dtypes(include=['number']).columns
        if len(numeric_cols) == 0:
            raise ValueError("출고 건수와 관련된 숫자형 컬럼을 찾을 수 없습니다.")

        # 출고 건수 합산
        blood_summary = blood_matched.groupby('clean_hospital')[numeric_cols].sum()

        # 결과 출력
        print("\n중복된 기관명과 해당 병원의 총 출고 건수:")
        for hospital, row in blood_summary.iterrows():
            total_count = row.sum()
            print(f"{hospital}: 총 출고 건수 {int(total_count)}개")
    else:
        print("\n중복된 기관명과 일치하는 병원의 혈액팩 방출 기록이 없습니다.")

except Exception as e:
    print(f"오류 발생: {e}")

서울 병원 데이터에서 중복된 기관명 수: 1872개

중복된 기관명과 해당 병원의 총 출고 건수:
굿모닝정형외과의원: 총 출고 건수 333개
노즈랩의원: 총 출고 건수 2개
대신의원: 총 출고 건수 151개
로앤산부인과의원: 총 출고 건수 5개
리영의원: 총 출고 건수 3개
미래의원: 총 출고 건수 3개
브라운성형외과의원: 총 출고 건수 2개
서울건우정형외과의원: 총 출고 건수 5개
서울본내과의원: 총 출고 건수 2개
서울삼성정형외과의원: 총 출고 건수 59개
서울외과의원: 총 출고 건수 2개
서울정형외과의원: 총 출고 건수 3개
서울제일신경외과의원: 총 출고 건수 10개
소망의원: 총 출고 건수 26개
아산진내과의원: 총 출고 건수 41개
연세내과의원: 총 출고 건수 1개
연세수내과의원: 총 출고 건수 12개
우리들병원: 총 출고 건수 8942개
우리연합의원: 총 출고 건수 6개
재단법인베스티안재단베스티안서울병원: 총 출고 건수 548개
주정형외과의원: 총 출고 건수 4개
참내과의원: 총 출고 건수 149개
참튼튼병원: 총 출고 건수 2195개
최내과의원: 총 출고 건수 10개
한마음의원: 총 출고 건수 24개


In [9]:
import pandas as pd

# 데이터 파일 경로
seoul_hospitals_path = "/content/drive/MyDrive/co-data/seoul_hospitals_selected.csv"
modified_seoul_hospitals_path = "/content/drive/MyDrive/co-data/modified_seoul_hospitals_selected.csv"

# 삭제할 병원 목록
hospitals_to_remove = [
    "굿모닝정형외과의원", "노즈랩의원", "대신의원", "로앤산부인과의원", "리영의원", "미래의원", "브라운성형외과의원",
    "서울건우정형외과의원", "서울본내과의원", "서울삼성정형외과의원", "서울외과의원", "서울정형외과의원", "서울제일신경외과의원",
    "소망의원", "아산진내과의원", "연세내과의원", "연세수내과의원", "우리들병원", "우리연합의원",
    "재단법인베스티안재단베스티안서울병원", "주정형외과의원", "참내과의원", "참튼튼병원", "최내과의원", "한마음의원"
]

# 병원 위경도 추가 목록
hospital_coordinates = {
    "우리들병원": (127.0496091, 37.51909256),
    "재단법인베스티안재단베스티안서울병원": (127.0280198, 37.56511028)
}

try:
    # 서울 병원 데이터 로드
    try:
        df = pd.read_csv(seoul_hospitals_path, encoding='utf-8')
    except UnicodeDecodeError:
        df = pd.read_csv(seoul_hospitals_path, encoding='cp949')

    # 컬럼명 표준화 (위도, 경도 컬럼 확인 및 수정)
    column_mapping = {
        '병원경도': '병원경도',
        '병원위도': '병원위도'
    }
    df.rename(columns=column_mapping, inplace=True)

    # 병원 목록에서 삭제
    df = df[~df['기관명'].isin(hospitals_to_remove)]

    # 위경도 추가
    for hospital, (longitude, latitude) in hospital_coordinates.items():
        df.loc[df['기관명'] == hospital, ['병원경도', '병원위도']] = longitude, latitude

    # 수정된 데이터 저장
    df.to_csv(modified_seoul_hospitals_path, index=False, encoding='utf-8-sig')
    print(f"수정된 데이터가 저장되었습니다: {modified_seoul_hospitals_path}")

except Exception as e:
    print(f"오류 발생: {e}")

수정된 데이터가 저장되었습니다: /content/drive/MyDrive/co-data/modified_seoul_hospitals_selected.csv


In [10]:
import pandas as pd

# 파일 경로 설정
seoul_hospitals_path = "/content/drive/MyDrive/co-data/modified_seoul_hospitals_selected.csv"
filtered_blood_release_path = "/content/drive/MyDrive/co-data/붙임 2. 정보공개 청구 회신자료 서식(병원별 혈액 청구량).xlsx"
final_output_path = "/content/drive/MyDrive/co-data/FINAL_BLOOD_RELEASE.xlsx"

# 병원 이름 매핑 딕셔너리 (기관명:출고병원)
hospital_name_mapping = {
    "강남세브란스병원": "연세대학교의과대학강남세브란스병원",
    "경희대학교병원(경희의료원)": "경희대학교병원",
    "고려대학교의료원안암병원": "학교법인고려중앙학원고려대학교의과대학부속병원(안암병원)",
    "고려대학교구로병원": "고려대학교의과대학부속구로병원",
    "서울대학교어린이병원": "서울대학교병원",
    "서울대학교암병원": "서울대학교병원",
    "서울성모병원": "학교법인가톨릭학원가톨릭대학교서울성모병원",
    "가톨릭대학교은평성모병원": "가톨릭대학교은평성모병원",
    "의료법인마리아병원": "의료법인마리아의료재단마리아병원",
    "서울아산병원": "재단법인아산사회복지재단서울아산병원",
    "연세의료원(신촌세브란스)": "연세대학교의과대학세브란스병원",
    "이화의대부속서울병원": "이화여자대학교의과대학부속서울병원",
    "이대부속목동병원": "이화여자대학교의과대학부속목동병원",
    "강동경희대학교병원": "강동경희대학교의대병원",
    "국립경찰병원": "경찰병원",
    "동부제일병원": "의료법인풍산의료재단동부제일병원",
    "동신병원": "의료법인동신의료재단동신병원",
    "서울의료원": "서울특별시서울의료원",
    "서울프라임병원(광진구)": "서울프라임병원",
    "성애병원": "성애의료재단성애병원",
    "양지병원": "의료법인서울효천의료재단에이치플러스양지병원"
}


# 위경도 수작업 추가 데이터
manual_coordinates = {
    "강북우리들병원": (127.0226341, 37.64254488),
    "우리들병원": (127.0496091, 37.51909256),
    "우리들병원(과해동)": (126.7963653099318, 37.56199597494205),
    "순천향대학교의과대학부속병원": (127.00441798640304, 37.53384172231443),
    "한국원자력의학원": (127.08269315796588, 37.62881598133035),
    "한림대 강동성심병원": (127.13526354631517, 37.53598408220376),
    "한강성심병원": (126.91033000120589, 37.52346674579277),
    "시립보라매병원": (126.92404876254014, 37.4937184),
    "강남성심병원": (126.9086725, 37.49324929),
    "의료법인 중앙의료재단 중앙병원": (127.03560992488869, 37.59688442926329),
    "참튼튼병원(동대문)": (127.0693591, 37.57273278),
    "참튼튼병원(노원)": (127.0636923, 37.65445005),
    "참튼튼병원": (126.9003936, 37.48213578),
    "서울백병원": (126.98884533055192, 37.56507317),
    "연세의료원 암병원": (126.94082769649863, 37.56211711412639),
    "의료법인성광의료재단(강남차)": (127.03466865188072, 37.506800025850865),
    "청구성심병원": (126.91955399169245, 37.62079154435882),
    "베스티안서울병원(성동구)": (127.0280198, 37.56511028),
    "강남성심(별관)": (126.9086725, 37.49324929),
    "서울병원(송파구)": (127.1247082, 37.50390523)
}

try:
    # 서울 병원 데이터 로드 (인코딩 처리)
    try:
        seoul_df = pd.read_csv(seoul_hospitals_path, encoding='utf-8')
    except UnicodeDecodeError:
        seoul_df = pd.read_csv(seoul_hospitals_path, encoding='cp949')

    # 초기 데이터 shape 확인
    print(f"서울 병원 데이터 크기: {seoul_df.shape}")

    # 혈액 방출 데이터 로드
    blood_df = pd.read_excel(filtered_blood_release_path)
    print(f"혈액 방출 초기 데이터 크기: {blood_df.shape}")

    # 컬럼 이름 확인 및 필요한 컬럼 추출
    print("\n원본 데이터 컬럼:")
    print(f"서울 병원 데이터 컬럼: {seoul_df.columns.tolist()}")
    print(f"혈액 방출 데이터 컬럼: {blood_df.columns.tolist()}")

    # 기관명과 출고병원 컬럼 확인
    institution_col = '기관명' if '기관명' in seoul_df.columns else next(
        (col for col in seoul_df.columns if '기관' in col or '병원' in col or '이름' in col or 'name' in col.lower()), None
    )

    hospital_col = '출고병원' if '출고병원' in blood_df.columns else next(
        (col for col in blood_df.columns if '병원' in str(col) or 'hospital' in str(col).lower() or '기관' in str(col)), None
    )

    release_date_col = '출고일자' if '출고일자' in blood_df.columns else next(
        (col for col in blood_df.columns if '일자' in str(col) or '날짜' in str(col) or 'date' in str(col).lower()), None
    )

    release_count_col = '출고건수' if '출고건수' in blood_df.columns else next(
        (col for col in blood_df.columns if '건수' in str(col) or '수량' in str(col) or 'count' in str(col).lower()), None
    )

    if institution_col is None or hospital_col is None or release_date_col is None or release_count_col is None:
        raise ValueError("필요한 컬럼을 찾을 수 없습니다.")

    print(f"\n사용할 컬럼 정보:")
    print(f"기관명 컬럼: {institution_col}")
    print(f"출고병원 컬럼: {hospital_col}")
    print(f"출고일자 컬럼: {release_date_col}")
    print(f"출고건수 컬럼: {release_count_col}")

    # 공백 제거 후 병원명 정리
    seoul_df['clean_name'] = seoul_df[institution_col].astype(str).str.replace(' ', '', regex=True)
    blood_df['clean_hospital'] = blood_df[hospital_col].astype(str).str.replace(' ', '', regex=True)

    # 병원 이름 매핑 적용
    for original, mapped in hospital_name_mapping.items():
        blood_df.loc[blood_df['clean_hospital'] == original.replace(' ', ''), 'clean_hospital'] = mapped.replace(' ', '')

    # 출고일자 형식 변환 (YYYY-MM-DD 형식으로)
    blood_df[release_date_col] = pd.to_datetime(blood_df[release_date_col]).dt.strftime('%Y-%m-%d')

    # 위경도 데이터 추가
    seoul_df = seoul_df[['clean_name', '병원경도', '병원위도']]

    # 병원 위경도를 병합
    merged_df = blood_df.merge(seoul_df, left_on='clean_hospital', right_on='clean_name', how='left')

    # 수작업 위경도 데이터 적용
    for hospital, coords in manual_coordinates.items():
        hospital_clean = hospital.replace(' ', '')
        mask = merged_df['clean_hospital'] == hospital_clean
        merged_df.loc[mask, '병원경도'] = coords[0]
        merged_df.loc[mask, '병원위도'] = coords[1]

    # 위경도 정보가 없는 병원 목록 저장
    missing_coords = merged_df[merged_df['병원경도'].isna() | merged_df['병원위도'].isna()]
    removed_hospitals = missing_coords[hospital_col].unique().tolist()

    # 위경도 정보가 없는 행 삭제
    final_df = merged_df.dropna(subset=['병원경도', '병원위도'])

    # 최종 데이터에서 필요한 컬럼만 선택
    final_df = final_df[[release_date_col, hospital_col, release_count_col, '병원경도', '병원위도']]

    # 컬럼명 변경
    final_df.columns = ['출고일자', '기관명', '출고건수', '병원경도', '병원위도']

    # 최종 데이터 저장
    final_df.to_excel(final_output_path, index=False)

    # 결과 출력
    print(f"\n최종 데이터가 {final_output_path}에 저장되었습니다.")
    print(f"초기 데이터 크기: {blood_df.shape}")
    print(f"최종 데이터 크기: {final_df.shape}")
    print(f"제거된 행 수: {blood_df.shape[0] - final_df.shape[0]}")

    # 제거된 병원 목록 출력
    if len(removed_hospitals) > 0:
        print("\n위경도 정보가 없어 제거된 병원 목록:")
        for hospital in removed_hospitals:
            print(f"- {hospital}")
    else:
        print("\n위경도 정보가 없는 병원이 없습니다.")

except Exception as e:
    print(f"오류 발생: {e}")
    import traceback
    traceback.print_exc()

서울 병원 데이터 크기: (21286, 3)
혈액 방출 초기 데이터 크기: (937409, 3)

원본 데이터 컬럼:
서울 병원 데이터 컬럼: ['기관명', '병원경도', '병원위도']
혈액 방출 데이터 컬럼: ['출고일자', '출고병원', '출고건수']

사용할 컬럼 정보:
기관명 컬럼: 기관명
출고병원 컬럼: 출고병원
출고일자 컬럼: 출고일자
출고건수 컬럼: 출고건수

최종 데이터가 /content/drive/MyDrive/co-data/FINAL_BLOOD_RELEASE.xlsx에 저장되었습니다.
초기 데이터 크기: (937409, 4)
최종 데이터 크기: (156469, 5)
제거된 행 수: 780940

위경도 정보가 없어 제거된 병원 목록:
- (성서본원)계명대학교 동산의료원
- (의) 한라의료재단 제주한라병원
- (칠곡)경북대학교병원
- BS종합병원
- CHA의과대학교분당차병원
- W(더블유)병원
- 가톨릭관동대학교 국제성모병원
- 가톨릭대학교 대전성모병원
- 가톨릭대학교 성빈센트병원
- 가톨릭대학교의정부성모병원
- 가톨릭대학교인천성모병원
- 강릉아산병원
- 강릉의료원
- 강원대학교병원
- 건양대학교병원
- 검단탑병원
- 경기도의료원 파주병원
- 경북대학교병원(본원)
- 경상국립대학교병원
- 경상북도 포항의료원
- 경주동국대학교병원
- 경주동산병원
- 계명대학교 대구동산병원
- 고려대부속안산병원
- 고령영생병원
- 고신대학교복음병원
- 광명성애병원
- 광주보훈병원
- 광주한국병원
- 광주한사랑병원
- 구병원(구의료재단)
- 구포성심병원
- 국립암센터
- 국민건강보험공단일산병원
- 군산의료원
- 굿모닝병원
- 김포우리병원
- 남양주한양병원
- 논산백제병원
- 단국대학교병원
- 대구가톨릭대학교의료원
- 대구보훈병원
- 대동병원
- 대전선병원
- 대전유성선병원
- 대전한국병원
- 동강병원
- 동국대학교일산불교병원
- 동남권원자력의학원
- 동래봉생병원
- 동수원병원
- 동아대학교병원
- 동의병원
- 동천동강병원(동강의료재단)
- 명지의료재단명지병원
- 모

In [11]:
import pandas as pd

# 파일 경로 설정
seoul_hospitals_path = "/content/drive/MyDrive/co-data/modified_seoul_hospitals_selected.csv"
filtered_blood_release_path = "/content/drive/MyDrive/co-data/붙임 2. 정보공개 청구 회신자료 서식(병원별 혈액 청구량).xlsx"
final_output_path = "/content/drive/MyDrive/co-data/FINAL_BLOOD_RELEASE.xlsx"

# 병원 이름 매핑 딕셔너리 (기관명:출고병원)
hospital_name_mapping = {
    "강남세브란스병원": "연세대학교의과대학강남세브란스병원",
    "경희대학교병원(경희의료원)": "경희대학교병원",
    "고려대학교의료원안암병원": "학교법인고려중앙학원고려대학교의과대학부속병원(안암병원)",
    "고려대학교구로병원": "고려대학교의과대학부속구로병원",
    "서울대학교어린이병원": "서울대학교병원",
    "서울대학교암병원": "서울대학교병원",
    "서울성모병원": "학교법인가톨릭학원가톨릭대학교서울성모병원",
    "가톨릭대학교은평성모병원": "가톨릭대학교은평성모병원",
    "의료법인마리아병원": "의료법인마리아의료재단마리아병원",
    "서울아산병원": "재단법인아산사회복지재단서울아산병원",
    "연세의료원(신촌세브란스)": "연세대학교의과대학세브란스병원",
    "이화의대부속서울병원": "이화여자대학교의과대학부속서울병원",
    "이대부속목동병원": "이화여자대학교의과대학부속목동병원",
    "강동경희대학교병원": "강동경희대학교의대병원",
    "국립경찰병원": "경찰병원",
    "동부제일병원": "의료법인풍산의료재단동부제일병원",
    "동신병원": "의료법인동신의료재단동신병원",
    "서울의료원": "서울특별시서울의료원",
    "서울프라임병원(광진구)": "서울프라임병원",
    "성애병원": "성애의료재단성애병원",
    "양지병원": "의료법인서울효천의료재단에이치플러스양지병원"
}

# 위경도 수작업 추가 데이터
manual_coordinates = {
    "강북우리들병원": (127.0226341, 37.64254488),
    "우리들병원": (127.0496091, 37.51909256),
    "우리들병원(과해동)": (126.7963653099318, 37.56199597494205),
    "순천향대학교의과대학부속병원": (127.00441798640304, 37.53384172231443),
    "한국원자력의학원": (127.08269315796588, 37.62881598133035),
    "한림대 강동성심병원": (127.13526354631517, 37.53598408220376),
    "한강성심병원": (126.91033000120589, 37.52346674579277),
    "시립보라매병원": (126.92404876254014, 37.4937184),
    "강남성심병원": (126.9086725, 37.49324929),
    "의료법인 중앙의료재단 중앙병원": (127.03560992488869, 37.59688442926329),
    "참튼튼병원(동대문)": (127.0693591, 37.57273278),
    "참튼튼병원(노원)": (127.0636923, 37.65445005),
    "참튼튼병원": (126.9003936, 37.48213578),
    "서울백병원": (126.98884533055192, 37.56507317),
    "연세의료원 암병원": (126.94082769649863, 37.56211711412639),
    "의료법인성광의료재단(강남차)": (127.03466865188072, 37.506800025850865),
    "청구성심병원": (126.91955399169245, 37.62079154435882),
    "베스티안서울병원(성동구)": (127.0280198, 37.56511028),
    "강남성심(별관)": (126.9086725, 37.49324929),
    "서울병원(송파구)": (127.1247082, 37.50390523)
}

try:
    # 서울 병원 데이터 로드 (인코딩 처리)
    try:
        seoul_df = pd.read_csv(seoul_hospitals_path, encoding='utf-8')
    except UnicodeDecodeError:
        seoul_df = pd.read_csv(seoul_hospitals_path, encoding='cp949')

    # 초기 데이터 shape 확인
    # print(f"서울 병원 데이터 크기: {seoul_df.shape}")

    # 혈액 방출 데이터 로드
    blood_df = pd.read_excel(filtered_blood_release_path)
    # print(f"혈액 방출 초기 데이터 크기: {blood_df.shape}")

    # 컬럼 이름 확인 및 필요한 컬럼 추출
    # print("\n원본 데이터 컬럼:")
    # print(f"서울 병원 데이터 컬럼: {seoul_df.columns.tolist()}")
    # print(f"혈액 방출 데이터 컬럼: {blood_df.columns.tolist()}")

    # 기관명과 출고병원 컬럼 확인
    institution_col = '기관명' if '기관명' in seoul_df.columns else next(
        (col for col in seoul_df.columns if '기관' in col or '병원' in col or '이름' in col or 'name' in col.lower()), None
    )

    hospital_col = '출고병원' if '출고병원' in blood_df.columns else next(
        (col for col in blood_df.columns if '병원' in str(col) or 'hospital' in str(col).lower() or '기관' in str(col)), None
    )

    release_date_col = '출고일자' if '출고일자' in blood_df.columns else next(
        (col for col in blood_df.columns if '일자' in str(col) or '날짜' in str(col) or 'date' in str(col).lower()), None
    )

    release_count_col = '출고건수' if '출고건수' in blood_df.columns else next(
        (col for col in blood_df.columns if '건수' in str(col) or '수량' in str(col) or 'count' in str(col).lower()), None
    )

    if institution_col is None or hospital_col is None or release_date_col is None or release_count_col is None:
        raise ValueError("필요한 컬럼을 찾을 수 없습니다.")

    # print(f"\n사용할 컬럼 정보:")
    # print(f"기관명 컬럼: {institution_col}")
    # print(f"출고병원 컬럼: {hospital_col}")
    # print(f"출고일자 컬럼: {release_date_col}")
    # print(f"출고건수 컬럼: {release_count_col}")

    # 공백 제거 후 병원명 정리
    seoul_df['clean_name'] = seoul_df[institution_col].astype(str).str.replace(' ', '', regex=True)
    blood_df['clean_hospital'] = blood_df[hospital_col].astype(str).str.replace(' ', '', regex=True)

    # 병원 이름 매핑 적용
    for original, mapped in hospital_name_mapping.items():
        blood_df.loc[blood_df['clean_hospital'] == original.replace(' ', ''), 'clean_hospital'] = mapped.replace(' ', '')

    # 출고일자 형식 변환 (YYYY-MM-DD 형식으로)
    blood_df[release_date_col] = pd.to_datetime(blood_df[release_date_col]).dt.strftime('%Y-%m-%d')

    # 위경도 데이터 추가
    seoul_df = seoul_df[['clean_name', '병원경도', '병원위도']]

    # 병원 위경도를 병합
    merged_df = blood_df.merge(seoul_df, left_on='clean_hospital', right_on='clean_name', how='left')

    # 수작업 위경도 데이터 적용
    for hospital, coords in manual_coordinates.items():
        hospital_clean = hospital.replace(' ', '')
        mask = merged_df['clean_hospital'] == hospital_clean
        merged_df.loc[mask, '병원경도'] = coords[0]
        merged_df.loc[mask, '병원위도'] = coords[1]

    # 위경도 정보가 없는 병원 목록 저장
    missing_coords = merged_df[merged_df['병원경도'].isna() | merged_df['병원위도'].isna()]
    removed_hospitals = missing_coords[hospital_col].unique().tolist()

    # 위경도 정보가 없는 행 삭제
    final_df = merged_df.dropna(subset=['병원경도', '병원위도'])

    # 최종 데이터에서 필요한 컬럼만 선택
    final_df = final_df[[release_date_col, hospital_col, release_count_col, '병원경도', '병원위도']]

    # 컬럼명 변경
    final_df.columns = ['출고일자', '기관명', '출고건수', '병원경도', '병원위도']

    # 최종 데이터 저장
    final_df.to_excel(final_output_path, index=False)

    # 결과 출력
    # print(f"\n최종 데이터가 {final_output_path}에 저장되었습니다.")
    # print(f"초기 데이터 크기: {blood_df.shape}")
    # print(f"최종 데이터 크기: {final_df.shape}")
    # print(f"제거된 행 수: {blood_df.shape[0] - final_df.shape[0]}")

    # 제거된 병원 목록 출력
    if len(removed_hospitals) > 0:
        print("\n위경도 정보가 없어 제거된 병원 목록:")
        for hospital in removed_hospitals:
            print(f"- {hospital}")

        print("\n위경도 정보가 없어 제거된 병원별 총 출고건수:")

        # 제거된 병원들의 총 출고건수 계산
        removed_hospitals_total = missing_coords.groupby(hospital_col)[release_count_col].sum().reset_index()

        # 출고건수가 큰 순서대로 정렬
        removed_hospitals_total = removed_hospitals_total.sort_values(release_count_col, ascending=False)

        # 결과 출력
        for idx, row in removed_hospitals_total.iterrows():
            print(f"- {row[hospital_col]}: {row[release_count_col]:,} 건")

        # 전체 제거된 출고건수 합계
        total_removed_count = removed_hospitals_total[release_count_col].sum()
        print(f"\n제거된 병원 전체 출고건수 합계: {total_removed_count:,} 건")
        print(f"전체 출고건수 대비 제거 비율: {(total_removed_count / blood_df[release_count_col].sum()) * 100:.2f}%")
    else:
        print("\n위경도 정보가 없는 병원이 없습니다.")

except Exception as e:
    print(f"오류 발생: {e}")
    import traceback
    traceback.print_exc()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
- 인천광역시의료원
- 인천기독병원
- 인천마디병원
- 인해의료재단 시원항병원
- 일곡양지요양병원
- 일산아르고요양병원
- 일산하이병원
- 자비의료재단 힘찬마디병원
- 자애요양병원
- 장유오즈산부인과
- 전주21세기병원
- 전주고려병원
- 전주나은요양병원
- 전주박천수정형외과
- 전주성모요양병원
- 전주한양병원
- 정다운 의료재단(나진요양병원)
- 정답병원
- 정상의료재단 바른병원
- 정읍사랑병원
- 정읍샘고을요양병원
- 정읍성신병원
- 정읍아산병원
- 정읍한서요양병원
- 제일조은병원(구.금왕태성병원)
- 제주선한병원
- 제주특별자치도 제주의료원
- 제천서울병원
- 조은손병원
- 중부요양병원
- 중앙병원(정안의료재단)
- 중앙효요양병원
- 지엔산부인과의원
- 진병원
- 진영병원(한서의료재단(구)진영삼성)
- 진주미래여성병원
- 진주세란병원
- 진천신라병원
- 진해연세에스병원
- 착한마디 병원
- 참내과의원
- 참사랑요양병원(의료법인청민의료재단)
- 참튼튼병원(의정부)
- 창문연합의원
- 창원메트로병원
- 창원병원
- 창원삼성요양병원
- 창원제일종합병원(석영의료재단)
- 창원한마음병원(동하의료재단)
- 창원힘찬병원
- 척시원병원
- 척척병원
- 천안21세기병원
- 천안우리병원
- 천안충무병원
- 첨단선병원
- 첨단우리병원
- 청담요양병원
- 청담참튼튼병원
- 청도대남병원
- 청라베스트요양병원
- 청아병원
- 청주프라임병원
- 청주현대병원
- 청파의료재단송탄중앙병원
- 추병원
- 춘천강남병원
- 춘천인성병원
- 충청남도공주의료원
- 충청남도천안의료원
- 충청남도홍성의료원
- 충청북도충주의료원
- 칠곡가톨릭병원
- 코끼리휴병원
- 큐병원
- 토마스의료재단 수원윌스기념병원
- 토마스의료재단(안양윌스기념병원)
- 튼튼정형외과병원
- 파주로하스요양병원
- 평택굿모닝병원
- 푸른병원
- 하남성심병원
- 하단프라임병원
- 한국병원
- 한국보훈복지의료공단부산보훈병원
- 한나여성병원
- 한빛병원
- 한신병원 (의)

## 2023년 데이터만 추출

In [12]:
import pandas as pd

# 엑셀 파일 경로
file_path = "/content/drive/MyDrive/co-data/FINAL_BLOOD_RELEASE.xlsx"

# 엑셀 파일 불러오기
df = pd.read_excel(file_path)

# '출고일자' 열을 날짜 형식으로 변환
df['출고일자'] = pd.to_datetime(df['출고일자'], errors='coerce')

# 2023년 데이터만 필터링
df_2023 = df[df['출고일자'].dt.year == 2023]

# 날짜 형식을 'YYYY-MM-DD' 형태로 변환
df_2023['출고일자'] = df_2023['출고일자'].dt.strftime('%Y-%m-%d')

# 결과 확인
print(df_2023.head())

# 2023년 데이터 저장
output_path = "/content/drive/MyDrive/co-data/FINAL_BLOOD_RELEASE_2023.xlsx"
df_2023.to_excel(output_path, index=False)

print(f"2023년 데이터가 {output_path}에 저장되었습니다.")

<ipython-input-12-574b8a39c50d>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2023['출고일자'] = df_2023['출고일자'].dt.strftime('%Y-%m-%d')


             출고일자             기관명  출고건수        병원경도       병원위도
91756  2023-01-01  가톨릭대학교 여의도성모병원    21  126.936731  37.518272
91757  2023-01-01   가톨릭대학교 은평성모병원    84  126.916150  37.633608
91758  2023-01-01          강남성심병원    16  126.908672  37.493249
91759  2023-01-01        강남세브란스병원    97  127.046313  37.492807
91760  2023-01-01       강동경희대학교병원    83  127.157085  37.552046
2023년 데이터가 /content/drive/MyDrive/co-data/FINAL_BLOOD_RELEASE_2023.xlsx에 저장되었습니다.
